In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.init as init

import numpy as np
import matplotlib.pyplot as plt
import copy
import random
from ast import Param
import json
import math

from utils.prune import apply_mask
from utils.count_improvement import improvements
from utils.normalize import normalize_weights
from utils.binary_ind import make_to_binary, modify_weights, apply_mask_binary
from  utils.active_weights import calculate_active_weights_percentage
from utils.save_model_txt import save_params

Proporciones iniciales de 1s antes de modificar:
Iteración inicial: 0.00% de pesos activos
Iteración 1: 10.00% de pesos activos
Iteración 2: 10.00% de pesos activos
Iteración 3: 10.00% de pesos activos
Iteración 4: 10.00% de pesos activos
Iteración 5: 10.00% de pesos activos
Iteración 6: 10.00% de pesos activos
Iteración 7: 10.00% de pesos activos
Iteración 8: 10.00% de pesos activos
Iteración 9: 10.00% de pesos activos
Iteración 10: 10.00% de pesos activos


In [2]:
# Verificar si la GPU está disponible y establecer el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [4]:
net = Net().to(device)
individuo = Net().to(device)

Cargamos los datos

In [5]:
# Definimos el transform para los datos de MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Cargamos el dataset de MNIST
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

# Definimos los DataLoaders para los conjuntos de entrenamiento y prueba
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False, num_workers=4)

In [6]:
# Definimos la función de pérdida para calcular el error
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
# train loop
train_loss = []
train_loss_mean = []
test_accuracies = []
n_individuo = 1
accuracy_threshold = 0.4

#Guardo el mejor individuo y la red original que tendrá que ser reestablecida cad epoca
individuo = make_to_binary(individuo)
best_individo_state_dict = individuo.state_dict()
net_state_dict = net.state_dict()

results = calculate_active_weights_percentage(individuo)
print("Total de parámetros:", results["total_params"])
print("Parámetros activos:", results["active_params"])
print("Porcentaje de parámetros activos: {:.2f}%".format(results["active_percentage"]))

#train loop
while True:
    net.load_state_dict(net_state_dict)
    if n_individuo == 1:
        print(f" --------------------- Individuo {n_individuo} --------------------- ")

        masked_net = apply_mask_binary(net, individuo)

        results = calculate_active_weights_percentage(masked_net)
        print("Total de parámetros:", results["total_params"])
        print("Parámetros activos:", results["active_params"])
        print("Porcentaje de parámetros activos: {:.2f}%".format(results["active_percentage"]))
          
        running_loss = 0.0
        # Train for 1 epoch
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            #optimizer.zero_grad()
            outputs = masked_net(inputs)
            loss = criterion(outputs, labels)
            #loss.backward()
            #optimizer.step()
            running_loss += loss.item()
        train_loss.append(running_loss / len(train_loader))

        masked_net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = masked_net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        #train_loss_mean.append(np.mean(train_loss))
        test_accuracies.append(accuracy)
        #print(f"Accuracy: {accuracy}, loss mean: {train_loss_mean[-1]}")
        print(f"Accuracy: {accuracy}", f"Loss: {train_loss[-1]}")
        best_loss = train_loss[-1]
        if accuracy > accuracy_threshold:
            break
        n_individuo += 1

        #Variamos el individuo desde el anterior para obtener uno nuevo
        nuevo_individuo = modify_weights(individuo)
        results = calculate_active_weights_percentage(nuevo_individuo)
        print("Total de parámetros ind:", results["total_params"])
        print("Parámetros activos:", results["active_params"])
        print("Porcentaje de parámetros activos: {:.2f}%".format(results["active_percentage"]))
    else:
        print(f" --------------------- Individuo {n_individuo} --------------------- ")

        masked_net = apply_mask_binary(net, nuevo_individuo)

        results = calculate_active_weights_percentage(masked_net)
        print("Total de parámetros:", results["total_params"])
        print("Parámetros activos:", results["active_params"])
        print("Porcentaje de parámetros activos: {:.2f}%".format(results["active_percentage"]))

        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = masked_net(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
        train_loss.append(running_loss / len(train_loader))

        masked_net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = masked_net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        test_accuracies.append(accuracy)
        print(f"Accuracy: {accuracy}", f"Loss: {train_loss[-1]}")
        if accuracy > accuracy_threshold:
            break

        # Verificamos que individuo tiene mejor desempeño
        if train_loss[-1] < best_loss:
            print("Nuevo mejor individuo")
            best_loss = train_loss[-1]
            best_individo_state_dict = nuevo_individuo.state_dict()
        
        # actualizamos el individuo
        individuo.load_state_dict(best_individo_state_dict)

        # Variamos el individuo desde el anterior para obtener uno nuevo
        nuevo_individuo = modify_weights(individuo)
        results = calculate_active_weights_percentage(nuevo_individuo)
        print("Total de parámetros ind:", results["total_params"])
        print("Parámetros activos:", results["active_params"])
        print("Porcentaje de parámetros activos: {:.2f}%".format(results["active_percentage"]))

        n_individuo += 1         


Total de parámetros: 235146
Parámetros activos: 23516
Porcentaje de parámetros activos: 10.00%
 --------------------- Individuo 1 --------------------- 
Total de parámetros: 235146
Parámetros activos: 23516
Porcentaje de parámetros activos: 10.00%
Accuracy: 0.0974 Loss: 2.302917439037803
Total de parámetros ind: 235146
Parámetros activos: 23514
Porcentaje de parámetros activos: 10.00%
 --------------------- Individuo 2 --------------------- 
Total de parámetros: 235146
Parámetros activos: 23514
Porcentaje de parámetros activos: 10.00%
Accuracy: 0.0974 Loss: 2.3029729623530213
Total de parámetros ind: 235146
Parámetros activos: 23514
Porcentaje de parámetros activos: 10.00%
 --------------------- Individuo 3 --------------------- 
Total de parámetros: 235146
Parámetros activos: 23514
Porcentaje de parámetros activos: 10.00%
Accuracy: 0.0974 Loss: 2.30301289644831
Total de parámetros ind: 235146
Parámetros activos: 23514
Porcentaje de parámetros activos: 10.00%
 --------------------- Ind

KeyboardInterrupt: 